# ITS Segmented Regression: Transit Ridership Analysis

**Author:** Tomasz Solis  
**Date:** December 2025  
**Learning Goal:** Practice ITS methodology on synthetic data with known ground truth

## What I'm Trying to Do

Measure the causal impact of express bus lanes launched January 1, 2024 using Interrupted Time Series analysis. Since EDA revealed non-parallel pre-trends across route types, I'm fitting separate models for Downtown, Suburban, and Cross-town routes rather than pooling.

This is practice for applying ITS at work (measuring product changes with no control group), so I'm focusing on:
1. Getting the technical implementation right
2. Understanding where assumptions break down
3. Validating against known true effects (advantage of synthetic data)
4. Documenting mistakes and learning from them

## Model Specification (Following The Mixtape Ch. 9)

For each route type separately:

```
ridership_t = β₀ + β₁(time) + β₂(post_intervention) + β₃(time_since_intervention) + ε_t
```

**What each parameter means:**
- `β₀`: Baseline level (intercept)
- `β₁`: Pre-intervention trend (how fast ridership was growing before express lanes)
- `β₂`: **Immediate level change** - the jump at intervention (this is what we care about!)
- `β₃`: **Slope change** - did growth accelerate/decelerate after intervention?

**Estimation approach:**
- OLS regression with Newey-West HAC standard errors (accounts for autocorrelation)
- No seasonal controls in this first attempt (will revisit if needed)

## Key Assumptions I'm Making

1. **No anticipation** - Ridership didn't change before January 2024 in anticipation of express lanes
2. **No other interventions** - Nothing else changed at the same time that would affect ridership
3. **Stable underlying process** - The relationship between time and ridership is stable

**Important note:** I'm NOT assuming parallel pre-trends (that was violated in EDA), which is why I'm doing segment-specific models.

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from scipy import stats
from datetime import datetime
from pathlib import Path

# Output directories
fig_output_dir = Path("../outputs/figures")
results_output_dir = Path("../outputs/results")
fig_output_dir.mkdir(parents=True, exist_ok=True)
results_output_dir.mkdir(parents=True, exist_ok=True)

# Plotting
pio.templates.default = "plotly_white"

ROUTE_COLORS = {
    'Downtown': '#1f77b4',
    'Suburban': '#ff7f0e',
    'Cross-town': '#2ca02c'
}

INTERVENTION_DATE = datetime(2024, 1, 1)

print("✓ Setup complete")

✓ Setup complete


---
## 1. Data Loading

In [2]:
# Load data
df = pd.read_csv('../data/easy_mode/transit_ridership_baseline.csv')
df['date'] = pd.to_datetime(df['date'])

# Ensure numeric types
df['time'] = pd.to_numeric(df['time'])
df['post_intervention'] = pd.to_numeric(df['post_intervention'])
df['time_since_intervention'] = pd.to_numeric(df['time_since_intervention'])
df['avg_ridership'] = pd.to_numeric(df['avg_ridership'])

print("="*60)
print("DATA CHECK")
print("="*60)
print(f"Total observations: {len(df):,}")
print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"\nPre-intervention:  {(df['post_intervention']==0).sum():3d} obs ({(df['post_intervention']==0).sum()/len(df)*100:.1f}%)")
print(f"Post-intervention: {(df['post_intervention']==1).sum():3d} obs ({(df['post_intervention']==1).sum()/len(df)*100:.1f}%)")
print(f"\nRoute types: {df['route_type'].unique().tolist()}")
print("="*60)

DATA CHECK
Total observations: 783
Date range: 2020-01-06 to 2024-12-30

Pre-intervention:  624 obs (79.7%)
Post-intervention: 159 obs (20.3%)

Route types: ['Cross-town', 'Downtown', 'Suburban']


---
## 2. Fit ITS Models

Building separate OLS models for each route type. Using Newey-West standard errors to account for autocorrelation (common in weekly time series).

**Design decision:** Starting without seasonal controls. If needed, I'll add them back, but want to see basic ITS mechanics first.

In [3]:
def fit_its_model(data, route_type, maxlags=4):
    """
    Fit ITS segmented regression for one route type.
    
    Parameters:
    -----------
    data : pd.DataFrame
        Full dataset
    route_type : str
        Which route to model
    maxlags : int
        Lags for Newey-West SE
    
    Returns:
    --------
    dict with model results and predictions
    """
    # Filter to this route
    route_data = data[data['route_type'] == route_type].copy()
    route_data = route_data.sort_values('time').reset_index(drop=True)
    
    # Dependent variable
    y = route_data['avg_ridership'].values
    
    # Independent variables - just the core ITS terms
    X = pd.DataFrame({
        'time': route_data['time'].values.astype(float),
        'post_intervention': route_data['post_intervention'].values.astype(float),
        'time_since_intervention': route_data['time_since_intervention'].values.astype(float)
    })
    
    # Add intercept
    X = sm.add_constant(X, has_constant='add')
    
    # Fit with Newey-West robust SE
    model = OLS(y, X.values)
    results = model.fit(cov_type='HAC', cov_kwds={'maxlags': maxlags})
    
    # Extract coefficients
    params = dict(zip(X.columns, results.params))
    se = dict(zip(X.columns, results.bse))
    
    beta_0 = params['const']
    beta_1 = params['time']
    beta_2 = params['post_intervention']
    beta_3 = params['time_since_intervention']
    
    se_2 = se['post_intervention']
    se_3 = se['time_since_intervention']
    
    # 95% confidence intervals
    ci_2 = (beta_2 - 1.96*se_2, beta_2 + 1.96*se_2)
    ci_3 = (beta_3 - 1.96*se_3, beta_3 + 1.96*se_3)
    
    # Fitted values
    fitted = results.fittedvalues
    
    # Counterfactual = project pre-trend forward (no treatment)
    X_counter = X.copy()
    X_counter['post_intervention'] = 0
    X_counter['time_since_intervention'] = 0
    counterfactual = results.predict(X_counter.values)
    
    return {
        'route_type': route_type,
        'model': results,
        'data': route_data,
        'fitted': fitted,
        'counterfactual': counterfactual,
        'coefficients': {
            'intercept': beta_0,
            'pre_trend': beta_1,
            'level_change': beta_2,
            'slope_change': beta_3
        },
        'standard_errors': {
            'level_change': se_2,
            'slope_change': se_3
        },
        'confidence_intervals': {
            'level_change': ci_2,
            'slope_change': ci_3
        },
        'r_squared': results.rsquared,
        'n_obs': len(route_data)
    }

# Fit all three models
print("="*60)
print("FITTING MODELS")
print("="*60)

models = {}
for route in ['Downtown', 'Suburban', 'Cross-town']:
    print(f"\nFitting {route}...")
    models[route] = fit_its_model(df, route, maxlags=4)
    print(f"  ✓ R² = {models[route]['r_squared']:.3f}")

print("\n" + "="*60)
print("✓ All models fitted")
print("="*60)

FITTING MODELS

Fitting Downtown...
  ✓ R² = 0.993

Fitting Suburban...
  ✓ R² = 0.989

Fitting Cross-town...
  ✓ R² = 0.983

✓ All models fitted


---
## 3. Results Summary

Key estimates with Newey-West robust standard errors.

In [4]:
# Build results table
results_list = []
for route in ['Downtown', 'Suburban', 'Cross-town']:
    m = models[route]
    results_list.append({
        'Route': route,
        'Pre-Trend (β₁)': f"{m['coefficients']['pre_trend']:.2f}",
        'Level Change (β₂)': f"{m['coefficients']['level_change']:.1f}",
        'β₂ 95% CI': f"[{m['confidence_intervals']['level_change'][0]:.1f}, {m['confidence_intervals']['level_change'][1]:.1f}]",
        'Slope Change (β₃)': f"{m['coefficients']['slope_change']:.2f}",
        'R²': f"{m['r_squared']:.3f}"
    })

results_df = pd.DataFrame(results_list)

print("="*60)
print("ITS MODEL RESULTS")
print("="*60)
print(results_df.to_string(index=False))

# Interpret in plain language
print("\n" + "="*60)
print("WHAT THIS MEANS")
print("="*60)

for route in ['Downtown', 'Suburban', 'Cross-town']:
    m = models[route]
    level = m['coefficients']['level_change']
    slope = m['coefficients']['slope_change']
    
    print(f"\n{route}:")
    print(f"  Immediate jump: {level:+.1f} riders when express lanes launched")
    
    if abs(slope) < 0.1:
        print(f"  Growth trend: Basically unchanged ({slope:+.2f}/week)")
    elif slope > 0:
        print(f"  Growth trend: Accelerated by {slope:+.2f} riders/week")
    else:
        print(f"  Growth trend: Slowed by {slope:.2f} riders/week")

print("\n" + "="*60)

# Save
results_df.to_csv(f"{results_output_dir}/its_results.csv", index=False)
print("\n✓ Saved to outputs/results/its_results.csv")

ITS MODEL RESULTS
     Route Pre-Trend (β₁) Level Change (β₂)      β₂ 95% CI Slope Change (β₃)    R²
  Downtown           2.47             307.7 [293.8, 321.6]             -0.29 0.993
  Suburban           1.66             202.5 [182.3, 222.7]             -0.12 0.989
Cross-town           1.09             150.7 [136.3, 165.0]              0.01 0.983

WHAT THIS MEANS

Downtown:
  Immediate jump: +307.7 riders when express lanes launched
  Growth trend: Slowed by -0.29 riders/week

Suburban:
  Immediate jump: +202.5 riders when express lanes launched
  Growth trend: Slowed by -0.12 riders/week

Cross-town:
  Immediate jump: +150.7 riders when express lanes launched
  Growth trend: Basically unchanged (+0.01/week)


✓ Saved to outputs/results/its_results.csv


---
## 4. Visualization: Actual vs Counterfactual

The key ITS visual: showing what actually happened vs what would have happened without express lanes.

In [5]:
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=[f'{r} Route' for r in ['Downtown', 'Suburban', 'Cross-town']],
    vertical_spacing=0.08
)

for i, route in enumerate(['Downtown', 'Suburban', 'Cross-town'], 1):
    m = models[route]
    data = m['data']
    
    # Actual ridership
    fig.add_trace(
        go.Scatter(
            x=data['date'],
            y=data['avg_ridership'],
            mode='lines',
            name=f'{route} (actual)' if i==1 else None,
            line=dict(color=ROUTE_COLORS[route], width=2),
            showlegend=(i==1),
            legendgroup='actual'
        ),
        row=i, col=1
    )
    
    # Counterfactual (post-period only)
    post_mask = data['post_intervention'] == 1
    post_dates = data.loc[post_mask, 'date']
    post_counter = m['counterfactual'][post_mask]
    
    fig.add_trace(
        go.Scatter(
            x=post_dates,
            y=post_counter,
            mode='lines',
            name='Counterfactual' if i==1 else None,
            line=dict(color='red', width=2, dash='dash'),
            showlegend=(i==1),
            legendgroup='counter'
        ),
        row=i, col=1
    )
    
    # Mark intervention
    fig.add_vline(
        x=INTERVENTION_DATE.timestamp() * 1000,
        line_dash="dot",
        line_color="gray",
        row=i, col=1
    )

fig.update_layout(
    height=900,
    title_text='ITS Results: Observed vs Counterfactual',
    showlegend=True
)

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Average Daily Ridership")

fig.show()
fig.write_image(f"{fig_output_dir}/05_its_counterfactual.png", scale=2)

print("\n✓ Saved to outputs/figures/05_its_counterfactual.png")


✓ Saved to outputs/figures/05_its_counterfactual.png


---
## 5. Treatment Effect Over Time

Calculating the difference (actual - counterfactual) for each week post-intervention.

In [6]:
fig = go.Figure()

for route in ['Downtown', 'Suburban', 'Cross-town']:
    m = models[route]
    data = m['data']
    
    # Post-intervention only
    post_mask = data['post_intervention'] == 1
    post_dates = data.loc[post_mask, 'date']
    post_actual = data.loc[post_mask, 'avg_ridership'].values
    post_counter = m['counterfactual'][post_mask]
    
    treatment_effect = post_actual - post_counter
    
    fig.add_trace(go.Scatter(
        x=post_dates,
        y=treatment_effect,
        mode='lines',
        name=route,
        line=dict(color=ROUTE_COLORS[route], width=2)
    ))

fig.add_hline(y=0, line_dash="dash", line_color="gray")

fig.update_layout(
    title='Treatment Effect Over Time',
    xaxis_title='Date',
    yaxis_title='Treatment Effect (riders)',
    height=500
)

fig.show()
fig.write_image(f"{fig_output_dir}/06_treatment_effect_time.png", scale=2)

# Average treatment effects
print("\n" + "="*60)
print("AVERAGE TREATMENT EFFECT (Post-Intervention)")
print("="*60)

for route in ['Downtown', 'Suburban', 'Cross-town']:
    m = models[route]
    data = m['data']
    
    post_mask = data['post_intervention'] == 1
    te = data.loc[post_mask, 'avg_ridership'].values - m['counterfactual'][post_mask]
    avg_te = te.mean()
    
    baseline = data.loc[~post_mask, 'avg_ridership'].mean()
    pct = (avg_te / baseline) * 100
    
    print(f"\n{route:12s}: {avg_te:+6.1f} riders ({pct:+5.1f}% vs baseline)")

print("\n" + "="*60)


AVERAGE TREATMENT EFFECT (Post-Intervention)

Downtown    : +300.3 riders (+40.5% vs baseline)

Suburban    : +199.3 riders (+35.5% vs baseline)

Cross-town  : +150.8 riders (+37.6% vs baseline)



---
## 6. Model Diagnostics

In [7]:
from statsmodels.stats.stattools import durbin_watson

print("="*60)
print("DIAGNOSTICS")
print("="*60)

for route in ['Downtown', 'Suburban', 'Cross-town']:
    m = models[route]
    dw = durbin_watson(m['model'].resid)
    
    print(f"\n{route}:")
    print(f"  R² = {m['r_squared']:.3f}")
    print(f"  Durbin-Watson = {dw:.2f}", end="")
    
    if dw < 1.5:
        print(" (positive autocorrelation - Newey-West SE handles this)")
    elif dw > 2.5:
        print(" (negative autocorrelation)")
    else:
        print(" (no significant autocorrelation)")

print("\n" + "="*60)
print("✓ Using Newey-West standard errors to account for autocorrelation")
print("="*60)

DIAGNOSTICS

Downtown:
  R² = 0.993
  Durbin-Watson = 1.56 (no significant autocorrelation)

Suburban:
  R² = 0.989
  Durbin-Watson = 1.11 (positive autocorrelation - Newey-West SE handles this)

Cross-town:
  R² = 0.983
  Durbin-Watson = 1.20 (positive autocorrelation - Newey-West SE handles this)

✓ Using Newey-West standard errors to account for autocorrelation


---
## 7. Validation: Did I Get It Right?

**Advantage of synthetic data:** I can check if my estimates match the true treatment effects that were built into the data.

The data was generated with:
- Downtown: +300 riders immediate jump, no slope change
- Suburban: +200 riders immediate jump, no slope change
- Cross-town: +150 riders immediate jump, no slope change

Let's see how close I got...

In [8]:
true_effects = {
    'Downtown': {'level': 300, 'slope': 0},
    'Suburban': {'level': 200, 'slope': 0},
    'Cross-town': {'level': 150, 'slope': 0}
}

print("="*60)
print("VALIDATION: My Estimates vs True Values")
print("="*60)

val_results = []
for route in ['Downtown', 'Suburban', 'Cross-town']:
    m = models[route]
    
    true_level = true_effects[route]['level']
    est_level = m['coefficients']['level_change']
    error = est_level - true_level
    pct_error = (error / true_level) * 100
    
    val_results.append({
        'Route': route,
        'True β₂': true_level,
        'My Estimate': f"{est_level:.1f}",
        'Error': f"{error:+.1f} ({pct_error:+.1f}%)",
        'In 95% CI?': 'Yes' if m['confidence_intervals']['level_change'][0] <= true_level <= m['confidence_intervals']['level_change'][1] else 'No'
    })

val_df = pd.DataFrame(val_results)
print("\n" + val_df.to_string(index=False))

# Check if validation passed
all_in_ci = all(r['In 95% CI?'] == 'Yes' for r in val_results)

print("\n" + "="*60)
if all_in_ci:
    print("✓ VALIDATION PASSED")
    print("\nAll true effects fall within my 95% confidence intervals.")
    print("This suggests the ITS methodology is working correctly.")
else:
    print("⚠️  VALIDATION FAILED")
    print("\nSome true effects are outside my confidence intervals.")
    print("This means either:")
    print("  1. Model specification is wrong")
    print("  2. I'm missing important controls")
    print("  3. Assumptions are violated")
print("="*60)

val_df.to_csv(f"{results_output_dir}/validation_results.csv", index=False)
print("\n✓ Saved to outputs/results/validation_results.csv")

VALIDATION: My Estimates vs True Values

     Route  True β₂ My Estimate        Error In 95% CI?
  Downtown      300       307.7 +7.7 (+2.6%)        Yes
  Suburban      200       202.5 +2.5 (+1.3%)        Yes
Cross-town      150       150.7 +0.7 (+0.4%)        Yes

✓ VALIDATION PASSED

All true effects fall within my 95% confidence intervals.
This suggests the ITS methodology is working correctly.

✓ Saved to outputs/results/validation_results.csv


---
## 8. What I Learned

### The Month Fixed Effects Trap

**Mistake I made first:** I initially included month fixed effects (January dummy, February dummy, etc.) to control for seasonality. This seemed logical - transit ridership has seasonal patterns.

**The problem:** The intervention happened on January 1, 2024. Every single post-intervention observation in January had BOTH:
- `post_intervention = 1` 
- `month_January = 1`

The model couldn't tell whether the jump was from express lanes or just "January seasonality." Result: Month fixed effects absorbed most of the treatment effect. My estimates were only 30% of true values.

**The fix:** Removed month controls for this baseline analysis. When intervention timing coincides with calendar periods, standard seasonal dummies create multicollinearity.

**Lesson for real work:**  I'll need to be careful about seasonal adjustment. Options:
1. Use Fourier terms (sin/cos) instead of month dummies
2. Detrend and deseasonalize before modeling  
3. Accept that some seasonal variation can't be separated from treatment

### What Worked Well

1. **Segment-specific models:** Handling non-parallel pre-trends by modeling each route separately was the right call
2. **Newey-West SE:** Accounts for autocorrelation without having to explicitly model it
3. **Validation:** Having ground truth to check against caught the month FE mistake immediately

### Assumptions That Held

- No anticipation effects (ridership stable before January 2024)
- Clear intervention point (no gradual rollout)
- Stable underlying relationship (R² > 0.85 for all routes)

### Limitations to Acknowledge

- This is "easy mode" synthetic data - real data will be messier
- No concurrent interventions to worry about here
- Known treatment effects aren't available with real data
- Can't test for spillovers between route types

---

## Next Steps

1. **Apply to realistic dataset** - smaller effects, more noise, confounders
2. **Robustness checks** - placebo tests with fake intervention dates
3. **Sensitivity analysis** - how do results change with different specifications?

---

## Key Takeaway

ITS can work even with non-parallel pre-trends IF you model segments separately. The bigger challenge will be:
1. Distinguishing treatment from concurrent events
2. Communicating uncertainty honestly
3. Avoiding the seasonal adjustment trap I fell into here

Good practice for the real thing.